In [2]:

from utils.logging_utils import create_logger
from yaml import safe_load
from pathlib import Path
from ingestion.database import VectorDataBase, PostgresDatabase
from utils.logging_utils import create_logger
from preprocessing.spacy_prep import SpacyPrep
from modeling_clusterization.embedding import FastTextModel
from modeling_clusterization.gpt_models import ChatGpt, Gpt4All
from sessions.application_session import ApplicationSession
import os 

In [4]:
    
parent_folder_path = Path().cwd()
print(parent_folder_path)
data_path = parent_folder_path / 'data' 
app_questions_data_path = data_path / 'app_questions' 
embed_model_path = app_questions_data_path / 'artifacts' / 'fasttext.model'
fasttext_training_it_dataset_path = app_questions_data_path / 'prep_data' / 'prep_sentences_50K.csv'

# with open('src/universal_config.yaml', 'r') as file:
with open('universal_config.yaml', 'r') as file:

    cf = safe_load(file)

cf_db_p = cf['database']['postgres']
cf_db_q = cf['database']['qdrant']

# with open('src/training_pipeline_a_config.yaml', 'r') as file:
with open('training_pipeline_a_config.yaml', 'r') as file:

    cf_a = safe_load(file)

# with open('src/training_pipeline_b_config.yaml', 'r') as file:
with open('training_pipeline_b_config.yaml', 'r') as file:

    cf_b = safe_load(file)


# paths to gpt binaries and generated output csv files
cf_gpt_model_to_use = cf['gpt_model_to_use']
model_path = parent_folder_path / 'data' / cf_a['ingestion']['gpt_model_constants']['model_path']
gpt_model_str =  cf_a['ingestion']['gpt_model_constants'][cf_gpt_model_to_use]['gpt_model_str'] #or use gpt4all constants

# strings to generate prompts
system_string = cf_b['ingestion']['gpt_model_constants']['default_prompt_strings']['system_string']
user_string = cf_b['ingestion']['gpt_model_constants']['default_prompt_strings']['user_string']




# logging constants and pipeline logger
log_level = cf['utils']['log_level']
tpb_logger = create_logger(log_level, log_name = 'training_pipeline_b_log')
tpb_logger.info('running training_pipeline_b.py')

postgres_db = PostgresDatabase(cf_db_p['db_url'])

[08-02 00:56:24] {training_pipeline_b_log:42} INFO - running training_pipeline_b.py


d:\project\ai_job_autopilot\gitlab\ai_core\src


In [7]:
postgres_db.connect()

[08-02 00:57:44] {PostgresDatabase:264} INFO - Postgres database core_backend_stage_db has been connected 


In [34]:
from psycopg2.sql import SQL, Identifier

table_name_forms= "forms_auto_fill"
table_name_questions = "questions"
user_id = "b5f5f813-dafe-4cce-8f15-089bee4efacb"

query = SQL("""SELECT 
question_id, 
name
FROM %s  f
LEFT JOIN %s q ON q.id = f.question_id
WHERE f.user_id = '%s'
""")




In [35]:
query

SQL("SELECT \nquestion_id, \nname\nFROM %s  f\nLEFT JOIN %s q ON q.id = f.question_id\nWHERE f.user_id = '%s'\n")

In [36]:
postgres_db.cursor.execute(query, (table_name_forms,  table_name_questions,  user_id))
results = postgres_db.cursor.fetchall()

SyntaxError: syntax error at or near "'forms_auto_fill'"
LINE 4: FROM 'forms_auto_fill'  f
             ^


In [71]:
# https://stackoverflow.com/questions/46770201/compose-dynamic-sql-string-with-psycopg2


# You can use psycopg2.sql.Identifier to interpolate an identifier to a query, e.g.:

# from psycopg2.sql import Identifier, SQL

# query = SQL('SELECT * FROM {}.{}').format(*map(Identifier, (schema, table)))
# print(query.as_string(conn))
# cur.execute(query)


In [66]:
print(query.as_string(postgres_db.conn))
postgres_db.cursor.execute(query)
results = postgres_db.cursor.fetchall()

SELECT 
* 
FROM "POSTGRES_DB_NAME_STAGE"."questions"



InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [60]:
query = f"""SELECT 
question_id, 
name
FROM {table_name_forms} f
LEFT JOIN {table_name_questions} q ON q.id = f.question_id
WHERE f.user_id = '{user_id}'
"""

print(query)


SELECT 
question_id, 
name
FROM forms_auto_fill f
LEFT JOIN questions q ON q.id = f.question_id
WHERE f.user_id = 'b5f5f813-dafe-4cce-8f15-089bee4efacb'



In [2]:
import os
from typing import List
from yaml import safe_load
from pathlib import Path
import numpy as np



with open('../src/universal_config.yaml', 'r') as file:
    cf = safe_load(file)

cf_db = cf['database']['postgres']
database = cf_db['app_q_table']

In [3]:
try:
    conn = psycopg2.connect(
        host="localhost",
        dbname=os.getenv('POSTGRES_DB_NAME_STAGE'),
        user=os.getenv('POSTGRES_DB_USER_STAGE'),
        password=os.getenv('POSTGRES_DB_PASS_STAGE')
    )
except Exception as err:
    print(f"Connection error {err}")
print("Hi")



Hi
